# Supermarket Data Scraper

The purpose of this is to collect data around ingredients that are needed for the recipes that are being suggested, so the end user will have an idea around the cost of each ingredient and where that item could be ordered.

We are going to use https://tesco.com/groceries as the main data source for this exercise. 

To ensure we don't lose data and it can be fetched in batches, we will split this into two steps: First we fetch list of products we would like to scrape and store them in a CSV file. Then we fetch top X from that list that is not scraped, and do the data scraping on it and update the CSV file.

## Prepare Data

For this, we first look at the sitemap of the website to get list of products that we would like to scrape. Then we generate a CSV file with following columns:
* product_id
* product_name
* product_price
* product_price_details
* product_url

In [1]:
# Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
sitemap_url = 'https://www.tesco.com/groceries/sitemap/UK.en.pdp.sitemap.xml'
headers = {
        'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8'
    }
html = requests.get(sitemap_url, headers=headers).content
soup = BeautifulSoup(html, "xml")

In [3]:
# fetch all the locations and filter it to have only product pages
all_locs = soup.find_all('loc')
len(all_locs)

10500

In [4]:
products = [ { 
    'product_id': 0, 
    'product_name': 'N/A', 
    'product_price': 'N/A', 
    'product_price_details': 'N/A', 
    'product_url': i.get_text()
} for i in all_locs]
print('We found ', len(products), ' Products')
products[0]

We found  10500  Products


{'product_id': 0,
 'product_name': 'N/A',
 'product_price': 'N/A',
 'product_price_details': 'N/A',
 'product_url': 'https://www.tesco.com/groceries/en-GB/products/313732035'}

In [5]:
# Convert to dataframe
prd_df = pd.DataFrame(products)
prd_df.head()

,product_id,product_name,product_price,product_price_details,product_url
0,0,N/A,N/A,N/A,https://www.tesco.com/groceries/en-GB/products...
1,0,N/A,N/A,N/A,https://www.tesco.com/groceries/en-GB/products...
2,0,N/A,N/A,N/A,https://www.tesco.com/groceries/en-GB/products...
3,0,N/A,N/A,N/A,https://www.tesco.com/groceries/en-GB/products...
4,0,N/A,N/A,N/A,https://www.tesco.com/groceries/en-GB/products...


In [6]:
prd_df.to_csv('supermarket-data.csv')

## Scrape the data

We have list of products in the CSV file, so here we can check the records that are not updated and fetch their data from the website

In [7]:
import pandas as pd
import time
from getpass import getpass
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import pathlib
import os
from os.path import join

import warnings
warnings.filterwarnings('ignore')

In [8]:
# Setup driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [9]:
def scrape_product(url):
    # Sleep for a short time, to avoid killing the server
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(3)
    product_name = driver.find_element(By.XPATH, '//h1[@data-auto="pdp-product-title"]').text
    prices = [i.text.strip() for i in driver.find_elements(By.XPATH, '//*[@data-auto="pdp-buy-box"]//p')]
    driver.quit()
    product_price = 'N/A'
    product_price_details = 'N/A'
    if len(prices) > 0:
        product_price = prices[0]
    if len(prices) > 1:
        product_price_details = prices[-1]
    return {
        'product_id' : url.split('/')[-1],
        'product_name': product_name,
        'product_price': product_price,
        'product_price_details': product_price_details,
        'product_url': url
    }

In [52]:
product_list = pd.read_csv('supermarket-data.csv')
product_list = product_list.drop(columns=[i for i in product_list.columns if i.find('Unnamed') > -1])
product_list.head()

,product_id,product_name,product_price,product_price_details,product_url
0,313732035,Foster's Lager Beer Can 18x440ml,£15.99,£2.02/litre,https://www.tesco.com/groceries/en-GB/products...
1,314598779,Celebrations Chocolate Tub 600G,£5.00,£0.83/100g,https://www.tesco.com/groceries/en-GB/products...
2,267158243,Tesco Smoked Boneless Gammon Joint,NaN,NaN,https://www.tesco.com/groceries/en-GB/products...
3,309196093,Oral-B Pro 3 Black Cross Action Electric Tooth...,£100.00,£100.00/each,https://www.tesco.com/groceries/en-GB/products...
4,260951547,Tesco Lamb Whole Shoulder Joint,£20.40,£10.20/kg,https://www.tesco.com/groceries/en-GB/products...


In [48]:
BATCH_SIZE = 10
def update_row(r):
    p = scrape_product(r['product_url'])
    return [int(p['product_id']),p['product_name'],p['product_price'],p['product_price_details'],p['product_url']]

In [49]:
# pick items to be updated
items_to_update = product_list[product_list['product_id'] <= 0].head(BATCH_SIZE)
items_to_update.shape

(10, 5)

In [50]:
items_to_update = items_to_update.apply(update_row, axis=1, result_type='expand')
items_to_update.columns = [i for i in product_list.columns]
items_to_update.head()

,product_id,product_name,product_price,product_price_details,product_url
1,314598779,Celebrations Chocolate Tub 600G,£5.00,£0.83/100g,https://www.tesco.com/groceries/en-GB/products...
2,267158243,Tesco Smoked Boneless Gammon Joint,N/A,N/A,https://www.tesco.com/groceries/en-GB/products...
3,309196093,Oral-B Pro 3 Black Cross Action Electric Tooth...,£100.00,£100.00/each,https://www.tesco.com/groceries/en-GB/products...
4,260951547,Tesco Lamb Whole Shoulder Joint,£20.40,£10.20/kg,https://www.tesco.com/groceries/en-GB/products...
5,314258952,TESCO 50PK SILVER DECORATION,£6.00,£0.12/each,https://www.tesco.com/groceries/en-GB/products...


In [51]:
# save result
def copy_changes(r):
    updated_df = items_to_update[items_to_update['product_url'] == r['product_url']]
    result_row = r
    if updated_df.shape[0] == 1:
        result_row = updated_df.iloc[0]
    return [result_row['product_id'],result_row['product_name'],result_row['product_price'],result_row['product_price_details'],result_row['product_url']]

product_list = product_list.apply(copy_changes, axis=1, result_type='expand')
product_list.columns = [i for i in items_to_update.columns]
product_list.head()
product_list.to_csv('supermarket-data.csv')